In [1]:
import requests
import pandas as pd
import datetime as dt

# CFPB Mortgage Complaints

In [2]:
class MortgageComplaints:
    def __init__(self, with_dates=True, with_narrative=False):
        self._dates_narrs = [with_dates, with_narrative]
        self.start_date = dt.date.today() - dt.timedelta(3) - dt.timedelta(365 * 3)
        self.end_date = dt.date.today() - dt.timedelta(2)
        self.url = self._make_url(with_dates, with_narrative)

    def _make_url(self, with_dates, with_narrative):
        product_name = "Mortgage"
        result_max = 10000
        optional_type = "&tab=list"
        sort_by = "created_date_desc"
        fields = "all"
        if with_dates:
            url = f"""https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/?date_received_max={self.end_date}&date_received_min={self.start_date}\
&field={fields}&product={product_name}&size={result_max}&sort={sort_by}{optional_type}"""
        else:
            start_date = "2011-12-01"
            url = f"""https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/?date_received_max={self.end_date}&date_received_min={start_date}\
&field={fields}&product={product_name}&size={result_max}&sort={sort_by}{optional_type}"""
        if with_narrative:
            url += "&has_narrative=true"
        return url
    
    def setDates(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._dates_narrs[0] = True
        self.url = self._make_url(*self._dates_narrs)
        
    def getData(self):
        response = requests.get(self.url)
        data = response.json()
        df = pd.DataFrame([y['_source'] for y in data["hits"]["hits"]])
        return df 

In [3]:
mc = MortgageComplaints()

### By date chunks

In [4]:
start_stops = [
    # 2019
    ("2019-01-01", "2019-03-31"),
    ("2019-04-01", "2019-06-30"),
    ("2019-07-01", "2019-09-30"),
    ("2019-10-01", "2019-12-31"),
    # 2020
    ("2020-01-01", "2020-03-31"),
    ("2020-04-01", "2020-06-30"),
    ("2020-07-01", "2020-09-30"),
    ("2020-10-01", "2020-12-31"),
    # 2021
    ("2021-01-01", "2021-03-31"),
    ("2021-04-01", "2021-06-30"),
    ("2021-07-01", "2021-09-30"),
    ("2021-10-01", "2021-12-31"),
    # 2022
    ("2022-01-01", "2022-03-31"),
    ("2022-04-01", "2022-06-30"),
    ("2022-07-01", "2022-09-30"),
    ("2022-10-01", "2022-12-31"),
    # 2023
    ("2023-01-01", "2023-03-31"),
    ("2023-04-01", "2023-06-30"),
    ("2023-07-01", "2023-09-30"),
    ("2023-10-01", "2023-12-31"),
]

df = pd.DataFrame()

i = 1
for start_stop in start_stops:
    mc.setDates(*start_stop)
    temp = mc.getData()
    # ensure results are under max result count
    print(f"Q{i} {start_stop[0][:4]}:", len(temp), len(temp) <= 10000)
    df = pd.concat([df, temp])
    i += 1
    if i > 4:
        i = 1

Q1 2019: 5532 True
Q2 2019: 5795 True
Q3 2019: 5904 True
Q4 2019: 5473 True
Q1 2020: 5797 True
Q2 2020: 6114 True
Q3 2020: 6554 True
Q4 2020: 6190 True
Q1 2021: 7436 True
Q2 2021: 7018 True
Q3 2021: 6108 True
Q4 2021: 5972 True
Q1 2022: 6935 True
Q2 2022: 5861 True
Q3 2022: 5524 True
Q4 2022: 4968 True
Q1 2023: 7180 True
Q2 2023: 5248 True
Q3 2023: 5111 True
Q4 2023: 5319 True


### Clean df

In [5]:
df.drop(columns=["product", "has_narrative", "submitted_via", "consumer_disputed"], inplace=True)
df['complaint_what_happened'] = df.complaint_what_happened.replace("", None)
df['zip_code'] = df.zip_code.astype('str')

### Save data

In [6]:
df.to_excel('./raw_data/MortgageComplaints.xlsx', index=False)